Authenticating using Notebook auth...
✓ Previously created authentication credentials were found.


RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [1]:
import ee
import google
import os
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AVHEtk4CNK_NNMH8JOKYytqiHaYeKDf-qISK2lJbn9OIf66m3mk7vCd-Acs

Successfully saved authorization token.


In [2]:
import ee_jupyter


In [3]:
import altair as alt
from ee_jupyter.colab import set_colab_output_cell_height
from ee_jupyter.ipyleaflet import Map
from ee_jupyter.ipyleaflet import Inspector
from ee_jupyter.layout import MapWithInspector
import ipyleaflet
import ipywidgets as widgets
from IPython.display import HTML
import math
import pandas as pd
from pprint import pprint  # for pretty printing

In [4]:
location_lonlat= ( 75.979404,11.798852)
map_init_params = {
    'center': list(reversed(location_lonlat)), # <lat,lon> ordering
    'zoom': 10
}
map1=Map(**map_init_params)
map1

Map(center=[11.798852, 75.979404], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [5]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

In [6]:
point=ee.Geometry.Point( 75.979404,11.798852)
landsat_rgb_viz={'bands':['B4','B3','B2'],'min':0.0,'max':0.3}
temporalFiltered=landsat8.filterDate('2016-01-01','2016-12-31')
spacialFiltered=temporalFiltered.filterBounds(point)
mediana=temporalFiltered.median()
map2=Map(**map_init_params)
map2.addLayer(mediana,landsat_rgb_viz)
map3=Map(**map_init_params)
spatialFiltered1 = landsat8.filterBounds(point)
temporalFiltered1 = spatialFiltered1.filterDate('2016-01-01', '2016-12-31')
least_cloudy_image=temporalFiltered1.sort('CLOUD_COVER').first()
print(type(least_cloudy_image))
map3.addLayer(least_cloudy_image,landsat_rgb_viz)
widgets.VBox([map2,map3])


<class 'ee.image.Image'>


In [7]:
map3

Map(center=[11.798852, 75.979404], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [8]:
def add_ndvi(image):
    ndvi=image.normalizedDifference(['B5','B4']).rename('NDVI')
    return image.addBands(ndvi)

In [9]:
image_collection=landsat8.filterDate('2016-01-01','2016-12-31').filterBounds(point)
xy=image_collection.map(add_ndvi)
ndvi_vis_arams={'bands':'NDVI','min':-1,'max':1,'palette':['blue','yellow','green']}
map5=Map(**map_init_params)
map5.addLayer(xy.qualityMosaic('NDVI'),ndvi_vis_arams)
map5

Map(center=[11.798852, 75.979404], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [17]:
imageset=xy.qualityMosaic('NDVI')
imageset.getDownloadURL()

EEException: Unable to export unbounded image.